In [ ]:
import numpy as np
import tensorflow as tf
import keras

**Loading the data using Kaggle API**





In [ ]:
#Install kaggle library
!pip install kaggle
#Make a directory called .kaggle which makes it invisible
!mkdir ~/.kaggle 
import json
token = {"username":"--name--","key":"--key---"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
   
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!mkdir data
!kaggle datasets download kmader/skin-cancer-mnist-ham10000 -p data
!apt install unzip
!mkdir HAM10000_images_part_1 
!mkdir HAM10000_images_part_2
!unzip /content/data/skin-cancer-mnist-ham10000.zip -d /content

!unzip /content/data/HAM10000_images_part_1.zip -d HAM10000_images_part_1 
!unzip /content/data/HAM10000_images_part_2.zip -d HAM10000_images_part_2

!echo files in /content/data: `ls data | wc -l`

**Format of the data set**
1.   Two Folders of Images
2.   csv file containing classes of all images
3.   There are 7 classes 

In [14]:
import os 
import errno
base_dir = 'base_dir'
image_class = ['nv','mel','bkl','bcc','akiec','vasc','df']
#3 folders are made: base_dir, train_dir and val_dir
try:
    os.mkdir(base_dir)
    
except OSError as exc:
    if exc.errno != errno.EEXIST:
        raise
    pass
train_dir = os.path.join(base_dir, 'train_dir')
try:
  os.mkdir(train_dir)
except OSError as exc:
    if exc.errno != errno.EEXIST:
        raise
    pass
val_dir = os.path.join(base_dir, 'val_dir')
try: 
  os.mkdir(val_dir)
  
except OSError as exc:
    if exc.errno != errno.EEXIST:
        raise
    pass
#make sub directories for the labels
for x in image_class:
      os.mkdir(train_dir+'/'+x)
for x in image_class:
      os.mkdir(val_dir+'/'+x)

In [ ]:
import pandas as pd
import shutil
df = pd.read_csv('/content/HAM10000_metadata.csv')

# Set y as the labels
y = df['dx']
#split data
from sklearn.model_selection import train_test_split
df_train, df_val = train_test_split(df, test_size=0.1, random_state=101, stratify=y)

# Transfer the images into folders, Set the image id as the index
image_index = df.set_index('image_id', inplace=True)

# Get a list of images in each of the two folders
folder_1 = os.listdir('HAM10000_images_part_1')
folder_2 = os.listdir('HAM10000_images_part_2')

# Get a list of train and val images
train_list = list(df_train['image_id'])
val_list = list(df_val['image_id'])

# Transfer the training images
for image in train_list:
  fname = image + '.jpg'
  if fname in folder_1:
    #the source path
    src = os.path.join('HAM10000_images_part_1', fname)
    #the destination path
    dst = os.path.join(train_dir+'/'+df['dx'][image], fname)
    print(dst)   
    shutil.copyfile(src, dst)
  if fname in folder_2:
    #the source path
    src = os.path.join('HAM10000_images_part_2', fname)
    #the destination path
    dst = os.path.join(train_dir, fname)
    shutil.copyfile(src, dst)

# Transfer the validation images
for image in val_list:
  fname = image + '.jpg'
  if fname in folder_1:
    #the source path
    src = os.path.join('HAM10000_images_part_1', fname)
    #the destination path
    dst = os.path.join(val_dir+'/'+df['dx'][image], fname)
  
    shutil.copyfile(src, dst)
        
  if fname in folder_2:
    #the source path
    src = os.path.join('HAM10000_images_part_2', fname)
    # destination path to image
    dst = os.path.join(val_dir, fname)
    # copy the image from the source to the destination
    shutil.copyfile(src, dst)


# **Preprocessing**

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import keras 
print(df.head())
image_class = ['nv','mel','bkl','bcc','akiec','vasc','df']
train_path = 'base_dir/train_dir/'
valid_path = 'base_dir/val_dir/'

image_shape = 224
train_datagen  = ImageDataGenerator(rescale=1./255)
val_datagen  = ImageDataGenerator(rescale=1./255)
#declares data generator for train and val batches
train_batches = train_datagen.flow_from_directory(train_path, 
                                                        target_size = (image_shape,image_shape),
                                                        classes = image_class,
                                                        batch_size = 64
                                                        )
valid_batches = val_datagen.flow_from_directory(valid_path, 
                                                        target_size = (image_shape,image_shape),
                                                        classes = image_class,
                                                        batch_size = 64                                                      )

# ***Training***

In [39]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten 
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
import tensorflow as tf
mobile = keras.applications.mobilenet.MobileNet()#transfer learning
x = mobile.layers[-6].output
# Add a dropout and dense layer for predictions
x = Dropout(0.25)(x)
predictions = Dense(7, activation='softmax')(x)
print(mobile.input)
net = Model(inputs=mobile.input, outputs=predictions)
mobile.summary()
for layer in net.layers[:-23]:
  layer.trainable = False
net.compile(optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy',tf.keras.metrics.TrueNegatives() , tf.keras.metrics.FalsePositives() , tf.keras.metrics.FalseNegatives() , tf.keras.metrics.TruePositives()])
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.00001)
history = net.fit_generator(train_batches, epochs=10)

Tensor("input_3:0", shape=(None, 224, 224, 3), dtype=float32)
Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
____________________________________________________

In [40]:
validation = net.evaluate(valid_batches)

9/9 [==============================] - 17s 2s/step - loss: 1.1687 - accuracy: 0.7620 - true_negatives_1: 3011.0000 - false_positives_1: 115.0000 - false_negatives_1: 132.0000 - true_positives_1: 389.0000


In [31]:
#saving the model 
net.save('skin-cancer-detection-mobilenet')

INFO:tensorflow:Assets written to: skin-cancer-detection-mobilenet/assets


In [25]:
#loading the model
model = keras.models.load_model('skin-cancer-detection-mobilenet')

In [ ]:
#HOW TO TEST ON SOME RANDOM IMAGE
import cv2
import numpy as np

img = cv2.imread('IMG_20200920_111508754.jpg')
img = cv2.resize(img,(224,224))
img = np.expand_dims(img,axis=0)
#assuming model is loaded already using load_model
print(np.argmax(model.predict(img)))